In [1]:
import re
from collections import defaultdict
def read_and_process_file(filename='input'):
    file = open(f'{filename}_22', 'r')
    lines = [l.rstrip() for l in file.readlines()]
    
    instructions = lines[-1]
    instructions = re.split('(\d+)',instructions)[:-1]
    instructions = [(instructions[i], int(instructions[i+1])) for i in range(0, len(instructions), 2)]
        
    
    num_cols = max(len(x) for x in lines[:-2])
    num_rows = len(lines[:-2])
    board = [[0 for c in range(num_cols)] for r in range(num_rows)]
    
    for r in range(num_rows):
        for i,c in enumerate(lines[r]):
            if c == '.':
                board[r][i] = 1
            elif c == '#':
                board[r][i] = 9
                
    return board, instructions

In [2]:
turning = {
    "" : 0,
    "R": 1,
    "L": -1
}

movement = {
    0: (0,1),
    2: (0,-1),
    3: (-1,0),
    1: (1,0)
}

printable = {
    0: ">",
    1: "v",
    2: "<",
    3: "ˆ"
}

In [3]:
def get_jump_cells_and_walls(board):
    limits = {}
    walls = set()
    num_rows = len(board)
    num_cols = len(board[0])
    
    for r in range(num_rows):
        non_empty_idx = [i for i,v in enumerate(board[r]) if v>0]
        limits[(r,min(non_empty_idx)),2] = ((r,max(non_empty_idx)),2)
        limits[(r,max(non_empty_idx)),0] = ((r,min(non_empty_idx)),0)
        
    for c in range(num_cols):
        non_empty_idx = [i for i,v in enumerate(board) if v[c]>0]
        limits[(min(non_empty_idx), c),3] = ((max(non_empty_idx), c),3)
        limits[(max(non_empty_idx), c),1] = ((min(non_empty_idx), c),1)
        
    for r in range(num_rows):
        for c in range(num_cols):
            if board[r][c] == 9:
                walls.add((r,c))
        
    return limits, set(walls)

def get_start_id(board):
    return (0,min([c for c in range(len(board[0])) if board[0][c] == 1]))

In [4]:
def calculate_result(position, direction):
    return 1000*(position[0]+1) + 4*(position[1]+1) + direction

In [5]:
def get_position_after_step(position, direction, limits, walls):
    _r, _c = position
    new_position = None
    current_move = movement[direction]

    limit_key = (position, direction)
    
    debug = False
    
    if debug:    
        print(f"Position: {position} | direction: {direction} | movement: {current_move}")
        print(f"Key: {limit_key}")
        
    #check if jump to another side
    if limit_key in limits:
        if debug:
            print(f'Jump to another side (direction: {direction})')
        new_position, new_direction = limits[limit_key]
    else:
        if debug:    
            print(f"Normal step")
        new_position, new_direction = (_r+current_move[0],_c+current_move[1]), direction
     
    #if new position is not a stone return it, otherwise do not move
    if new_position in walls:
        return position, direction, True
    else:
        return new_position, new_direction, False

In [6]:
def explore_board(board, instructions, limits, walls):
    current_position = get_start_id(board)
    current_direction = 0
    for current_instruction in instructions:
        current_direction = (current_direction + turning[current_instruction[0]])%4
        for step in range(current_instruction[1]):
            new_position, new_direction, blocked = get_position_after_step(current_position,current_direction, limits, walls)
            if blocked:
                break
            else:
                #print(f"New position: {new_position} | Direction: {current_direction}")
                board[new_position[0]][new_position[1]] = printable[current_direction]
            current_position = new_position
            current_direction = new_direction

    res = calculate_result(current_position, current_direction)
    print(f"Stopped at {(current_position[0]+1, current_position[1]+1)} and the direction is {current_direction}")
    print(f"The final value after exploring the board is {res}")
    return res

#### One star

In [7]:
board, instructions = read_and_process_file('input')
limits, walls = get_jump_cells_and_walls(board)
res = explore_board(board, instructions, limits, walls)

Stopped at (13, 141) and the direction is 2
The final value after exploring the board is 13566


#### Two stars

In [9]:
def detect_cubes(board, cube_size):
    cube_id = 1
    cubes = {}
    b = board.copy()
    current_pos = get_start_id(b)
    while len(cubes)<6:
        if board[current_pos[0]][current_pos[1]]!=0:
            cubes[cube_id] = current_pos
            cube_id+=1
        while ((current_pos[1])+cube_size < len(board[0])):
            new_pos = (current_pos[0],current_pos[1]+cube_size)
            if (board[new_pos[0]][new_pos[1]]!=0):
                cubes[cube_id] = new_pos
                cube_id+=1
            current_pos = new_pos
        current_pos = (current_pos[0]+cube_size, 0)
    return cubes

In [10]:
def calculate_cube_neighbors(cubes, cube_size):
    neighbors = defaultdict(dict)
    for cube_id in range(1,7):
        _cur = cubes[cube_id]
        for n_id in range(cube_id+1, 7):
            _next = cubes[n_id]
            if (_cur[0]==_next[0]) and (_cur[1]+cube_size == _next[1]):
                neighbors[cube_id][0] = n_id
                neighbors[n_id][2] = cube_id

            if (_cur[1]==_next[1]) and (_cur[0]+cube_size == _next[0]):
                neighbors[cube_id][1] = n_id
                neighbors[n_id][3] = cube_id
    for _dir in (0,1,2,3):
        for _cur in range(1,7):
            if _dir in neighbors[_cur]:
                # L connections
                _joiner = neighbors[_cur][_dir]
                _joiner_incoming_dir = [k for k,v in neighbors[_joiner].items() if v==_cur][0]

                incoming_shifts = (1,-1)
                for _shift in incoming_shifts:
                    _joiner_outgoing_dir = (_joiner_incoming_dir+_shift)%4
                    if _joiner_outgoing_dir in neighbors[_joiner]:
                        _tail = neighbors[_joiner][_joiner_outgoing_dir]
                        _tail_incoming_dir = [k for k,v in neighbors[_tail].items() if v==_joiner][0]

                        neighbors[_cur][(_dir + (-1)*_shift)%4] = _tail
                        neighbors[_tail][(_tail_incoming_dir + _shift)%4] = _cur

    return neighbors

In [11]:
def get_jump_cells_and_walls_for_a_cube(board, cubes, neighbors, cube_size):
    limits = []
    walls = set()
    num_rows = len(board)
    num_cols = len(board[0])
    
    cubes_limits = {k:{"r_s": v[0], "r_e": v[0]+cube_size-1, "c_s": v[1], "c_e": v[1]+cube_size-1} for k,v in cubes.items()}
    all_ranges = {}
    for k,v in cubes_limits.items():
        all_ranges[k] = {0: [(r,v["c_e"]) for r in range(v["r_s"], v["r_e"]+1)],
                  1: [(v["r_e"],c) for c in range(v["c_e"], v["c_s"]-1, -1)],
                  2: [(r,v["c_s"]) for r in range(v["r_e"], v["r_s"]-1, -1)],
                  3: [(v["r_s"],c) for c in range(v["c_s"], v["c_e"]+1)],
                     }
    
    for cube_id in range(1,7):
        for _dir in range(4):
            neighbor = neighbors[cube_id][_dir]
            neighbor_dir = [k for k,v in neighbors[neighbor].items() if v==cube_id][0]
            
            _from = [(x,_dir)for x in all_ranges[cube_id][_dir]]
            _to = [(x,(neighbor_dir+2)%4)for x in all_ranges[neighbor][neighbor_dir]]
            jump_cells = list(zip(_from,reversed(_to)))
            limits.extend(jump_cells)
             
    limits = {pair[0]:pair[1] for pair in limits}
        
    for r in range(num_rows):
        for c in range(num_cols):
            if board[r][c] == 9:
                walls.add((r,c))
        
    return limits, set(walls)

In [12]:
cube_size = 50
board, instructions = read_and_process_file('input')
cubes = detect_cubes(board, cube_size)
neighbors = calculate_cube_neighbors(cubes, cube_size)


limits, walls = get_jump_cells_and_walls_for_a_cube(board, cubes, neighbors, cube_size)
res = explore_board(board, instructions, limits, walls)

Stopped at (11, 112) and the direction is 3
The final value after exploring the board is 11451
